0907 목요일

In [1]:
from sqlalchemy.orm import declarative_base
# declarative_base는 SQLAlchemy ORM에서 제공하는 함수로, 데이터베이스 테이블과 Python 클래스를 매핑해주는 기반 클래스를 생성
base = declarative_base() #base 인스턴스를 생성
base.metadata.tables # <- Core Table 객체-RDMBS 
#.tables는 이 메타데이터 내의 모든 테이블에 대한 정보를 담고 있는 딕셔너리
base.registry # <- Object 등록 
#registry는 base에 등록된 모든 클래스에 대한 정보를 담고 있는 곳

In [3]:
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text

base.metadata.tables
#'USER'라는 이름의 테이블을 메타데이터에서 제거하는 코드
base.registry.dispose() # base에 등록된 모든 클래스 정보를 제거합니다.

In [4]:
class User(base):
    __tablename__ = 'USER'
    # obj->테이블이름   => DBMS 사용될 Table이름
    pk = Column('PK', Integer, primary_key=True)
    # obj->pk        => DBMS 해당 Table 사용되는 Column
    name = Column('NAME', Text)
    
    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}'

In [6]:
user1 = User(name='아무거나')
type(user1), type(base.metadata.tables['USER'])

(__main__.User, sqlalchemy.sql.schema.Table)

In [7]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///:memory:', echo=True)
base.metadata.create_all(engine)

2023-09-10 14:14:25,965 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-10 14:14:25,966 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2023-09-10 14:14:25,967 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-10 14:14:25,968 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("USER")
2023-09-10 14:14:25,969 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-10 14:14:25,971 INFO sqlalchemy.engine.Engine 
CREATE TABLE "USER" (
	"PK" INTEGER NOT NULL, 
	"NAME" TEXT, 
	PRIMARY KEY ("PK")
)


2023-09-10 14:14:25,973 INFO sqlalchemy.engine.Engine [no key 0.00166s] ()
2023-09-10 14:14:25,975 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
# session => engine.connect().execute() => DBMS
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(engine)
sess = Session()
sess.add(user1)
user1

PK:None, NAME:아무거나

In [9]:
sess.commit()
user1

2023-09-10 14:15:08,393 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-10 14:15:08,399 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?)
2023-09-10 14:15:08,400 INFO sqlalchemy.engine.Engine [generated in 0.00152s] ('아무거나',)
2023-09-10 14:15:08,403 INFO sqlalchemy.engine.Engine COMMIT
2023-09-10 14:15:08,408 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-10 14:15:08,415 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-10 14:15:08,416 INFO sqlalchemy.engine.Engine [generated in 0.00127s] (1,)


PK:1, NAME:아무거나

In [10]:
sess.is_modified(user1)

False

In [11]:
sess.dirty

IdentitySet([])

In [12]:
user1.name = '다른값'
user1

PK:1, NAME:다른값

In [13]:
sess.dirty

IdentitySet([PK:1, NAME:다른값])

In [14]:
sess.is_modified(user1)

True

In [15]:
sess.commit()

2023-09-10 14:15:46,906 INFO sqlalchemy.engine.Engine UPDATE "USER" SET "NAME"=? WHERE "USER"."PK" = ?
2023-09-10 14:15:46,907 INFO sqlalchemy.engine.Engine [generated in 0.00114s] ('다른값', 1)
2023-09-10 14:15:46,909 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
user1

2023-09-10 14:15:53,148 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-10 14:15:53,149 INFO sqlalchemy.engine.Engine SELECT "USER"."PK" AS "USER_PK", "USER"."NAME" AS "USER_NAME" 
FROM "USER" 
WHERE "USER"."PK" = ?
2023-09-10 14:15:53,150 INFO sqlalchemy.engine.Engine [cached since 44.74s ago] (1,)


PK:1, NAME:다른값

In [17]:
sess.add_all([User(name='2'), User(name='3')])
sess.commit()

2023-09-10 14:16:17,565 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?) RETURNING "PK"
2023-09-10 14:16:17,568 INFO sqlalchemy.engine.Engine [generated in 0.00033s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('2',)
2023-09-10 14:16:17,570 INFO sqlalchemy.engine.Engine INSERT INTO "USER" ("NAME") VALUES (?) RETURNING "PK"
2023-09-10 14:16:17,572 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('3',)
2023-09-10 14:16:17,574 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
userList = sess.query(User).all()
len(userList)
user1 is userList[0]
userList[1].name = '다른값2'
sess.dirty
sess.commit()
userList[1]

In [ ]:
[user for user in userList if user.name == '3'][0] is userList[2]
sess.query(User).where(User.name == '3').one() is userList[2]

class Address(base):
    __tablename__ = 'ADDRESS'
    
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    fk = Column('FK', None, ForeignKey('USER.PK'))
    
    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}, FK:{self.fk}'